# Image to Image translation with Conditional Adversarial Networks



## Introduction

- CNN은 loss를 최소화 하는 방법으로 좋은 학습을 진행함
- 그러나 "어떠한 것을 최소화 해야하는지" 항상 정해줘야함
- 예를 들어, Euclidean distance를 최소화 하라고 지정하면 blurry results가 나옴
  - 이는 모든 plausible output을 평균내는 방법으로 최소화를 해버리기 때문임
  - 우리가 진짜 원하는 결과가 나오게끔 손실함수를 최소화하는 것은 어렵다  


![Image](https://taeoh-kim.github.io/img/img3.PNG)

$$Loss =\sum _{ x \in EveryPixel }^{  }{ \left\Vert GT(x) - Pred(x) \right\Vert  }$$

- 네트워크 관점에서는 너무나 당연하지만 사람 관점에서는 좀...
- VAE랑 비교해보면?
  - VAE는 데이터 분포를 찾아내는 확률적 접근 방법(수학적으로는 더 정확함(?))
  - Image에 적용하면 당연히 사진같이 안나온다
  - GAN은 실제같은 data를 만들어서 discriminator라는 개념도 추가하여 결과물이 더 그럴듯함
- Make the output indistinguishable from reality


논문에서는 기존의 GAN을 conditional setting을 사용하여 학습시킴. 이를 줄여서 cGAN(conditional Generative Adversarial Networks)라고 부르며 image-to-image translation에 최적화 시킴. 


![Imgur](https://i.imgur.com/3PTJSRd.png)


## cGAN Method

- 기존의 GAN은 noise $z$에서 이미지를 생성
- cGAN의 generator G는 인풋 이미지 $x$와 noise $z$를 가지고 이미지를 생성
- 이렇게 생성된 이미지를 가지고 discriminator D가 생성된 이미지인지 진짜 이미지인지 구분

![Imgur](https://i.imgur.com/mQXhlwh.png)


- cGAN의 objective function은 아래와 같음

$${ L }_{ cGAN }(G,D)={ \mathbb E_{x,y}}\left[ \log { D(x,y) }  \right] +{ \mathbb E_{x,z} }\left[ \log { \left( 1-D\left( x,G(x,z \right)  \right)  }  \right] $$

D의 입장에서는 data로부터 뽑은 sample x는 D(x)=1이 되고, G에 임의의 noise distribution으로부터 뽑은 input z 넣고 만들어진  sample에 대해서는 D(G(z))=0가 되도록 노력한다. 즉, D는 실수할 확률을 낮추기(mini) 위해 노력하고 반대로 G는 D가 실수할 확률을 높이기(max) 위해 노력하는데, 따라서 둘을 같이 놓고보면 "minimax two-player game or minimax problem"이라 할 수 있음.

- 다른 연구 결과에 따르면, GAN에 추가적인 L2 distance loss(reconstruction loss)를 더해주면 더 잘된다고함 -- 논문에서는 둘 다 실험했을 때, L1 distance loss가 더 잘나온다고 말하고 있음

$${L}_{L1}(G)={ \mathbb E_{x, y, z}[{ \left\Vert y-G(x,z) \right\Vert  }_{ 1 }] }  $$

- 최종적인 목적식은 아래와 같음

$${ G }^{ * }=arg\min _{ G }{ \max _{ D }{ { L }_{ cGAN }(G,D)+\lambda { L }_{ L1 }(G) }  } $$

- $z$가 없더라도 네트워크가 $x$에서 $y$로 매핑을 할 수는 있으나, deterministic한 결과물만 나옴
- 기존 방법론들은 $z$로 가우시안 노이즈를 주는 방법을 사용
- 실험해봤더니 별로 효과적이지 않음. G가 그냥 노이즈를 무시하는 방식으로 학습을 해버림
- 논문에서는 이 방법보다는 dropout의 형태로 노이즈를 줬다고 함


### Generator with skips

- 여기서 만들고자 하는 결과물은 "underlying structure"는 같음
- 이런 구조를 살리기 위해서 auto encoder에 skip connection을 추가
- Unet처럼 skip connection을 연결해줌

![Image](https://taeoh-kim.github.io/img/img4-2.PNG) 


![Imgur](https://i.imgur.com/LbPKsrH.png)

### PatchGAN

- 기존 GAN은 D가 이미지의 전체를 바라보게 됨
- 이렇게 되면 blurry한 결과물이 나오게 된다
- 이미지 전체가 아닌 patch 단위로 보고 판단하면 이러한 문제를 해결할 수 있음
- L1 distance loss를 사용하여 low-frequency정보는 이미 학습이 잘 됬지만, high-frequency 정보를 잘 generation하기 위해서는 이 방법 적용이 필요함
- 전체 이미지에 대해 적용하고, 결과값은 평균내서 적용
- N X N 단위로 패치를 변경하며 실험 진행
- 70 x 70일때 가장 샤프한 이미지를 뽑아냄. 물론 풀 이미지보다는 좀 부정확해서 지표는 좀 낮음

![Imgur](https://i.imgur.com/05koPUa.png)

### 네트워크 구조 

![Imgur](https://i.imgur.com/hhJzbqq.png)



## Experiments


- 실험은 1 step on D, 1 step on G 방식으로 진행
- 평가는 다음과 같은 방식으로 진행
  1. 주관적 평가 : 사람이 직접 discriminator가 됨(아마존 설문조사 시스템 이용)
  2. FCN-score : 분류 모델에 생성된 이미지를 집어 넣어 잘 분류하는지 확인

![Imgur](https://i.imgur.com/iGEh6jx.png)

- 위 결과를 보면 L1로스만 썼을 때 blurry한 결과가 나오는 것을 확인할 수 있음
- cGAN만 사용하면($\lambda=0$) low frequency가 좀 안잡히는것을 확인할 수 있음

![Imgur](https://i.imgur.com/LmZU8oV.png)

- 색만 봤을 때 L1만 사용하면 GT의 분포에서 가장 멀게 나타남(칙칙한 색들)
- cGAN이 실제 색 분포와 더 가까움


![Imgur](https://i.imgur.com/o4mXoUQ.png)

![Imgur](https://i.imgur.com/IQhyD5k.png)

![Imgur](https://i.imgur.com/2TODI2X.png)